# Text classification with an RNN

This is a text classification that uses recurrent neural network on the IMDB large movie review dataset for sentiment analysis.


_source_: https://www.tensorflow.org/text/tutorials/text_classification_rnn


Design of Bidirectional Model                                                         |                                       Design of Stack two or more LSTM layers
:------------------------------------------------------------------------------------:|:-----------------------------------------------------------------------------------------------------:
![bidirectional](https://www.tensorflow.org/text/tutorials/images/bidirectional.png)  |  ![layered_bidirectional](https://www.tensorflow.org/text/tutorials/images/layered_bidirectional.png)

In [1]:
import numpy as np

import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
tfds.disable_progress_bar()

## Helpers functions

In [3]:
def plot_history_metric(history, metric):
    """
    Plot metric of model train by epochs history.
    """
    plt.plot(history.history[metric])
    plt.plot(history.history['val_' + metric], '')
    plt.xlabel('Epochs')
    plt.ylabel(metric)
    plt.legend([metric, 'val_' + metric])

## Setup input pipeline

In [4]:
dataset, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True, shuffle_files=True)
train_dataset, test_dataset = dataset["train"], dataset["test"]

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset imdb_reviews downloaded and prepared to /home/fausto/tensorflow_datasets/imdb_reviews/plain_text/0.1.0. Subsequent calls will reuse this data.


2022-03-13 08:26:59.727599: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-03-13 08:26:59.728378: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-13 08:26:59.730263: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [5]:
train_dataset.element_spec

(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

#### Take one text example and its label. 

In [6]:
example_text, example_label = None, None
for text, label in train_dataset.take(1):
    example_text = text.numpy()
    example_label = label.numpy()

2022-03-13 08:26:59.815535: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-03-13 08:26:59.833440: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2599990000 Hz


__Text Example:__

In [7]:
print(example_text)

b"Oh yeah! Jenna Jameson did it again! Yeah Baby! This movie rocks. It was one of the 1st movies i saw of her. And i have to say i feel in love with her, she was great in this move.<br /><br />Her performance was outstanding and what i liked the most was the scenery and the wardrobe it was amazing you can tell that they put a lot into the movie the girls cloth were amazing.<br /><br />I hope this comment helps and u can buy the movie, the storyline is awesome is very unique and i'm sure u are going to like it. Jenna amazed us once more and no wonder the movie won so many awards. Her make-up and wardrobe is very very sexy and the girls on girls scene is amazing. specially the one where she looks like an angel. It's a must see and i hope u share my interests"


__Label Example:__

In [8]:
print(example_label)

1


## Shuffle the data for train and create batches of these (text, label) pairs

In [9]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [10]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [11]:
example_batch_texts, example_batch_labels = None, None,
for example, label in train_dataset.take(1):
    example_batch_texts = example.numpy()
    example_batch_labels = label.numpy()

__Batch Texts Example:__

In [12]:
print(example_batch_texts[:3])

[b"I have to say I was very curious on viewing this film, and it was considered a notorious disaster when released by 20th Century Fox in 1970. It has also popped up on several critics lists of bad films, and this only deepened an interest, as I just had to see what made this movie so bad.Upon seeing it, I think I have my answers. Although I will say it does make for curious viewing, the acting, direction, and script are so laughingly bad, that the supposed satire is completely missing. Racquel Welch seems to try to carry the film, but after the opening sequence of the sex-change operation, the film goes so far down hill that she cannot handle this task alone. John Huston as Uncle Buck Loner is certainly no help, as he licks and leers at the screen, he sometimes looks like he wonders himself what he's doing there. Rex Reed bounces around as Myron, Myra's alter ego, and even has his own celebrated masturbation scene. Bravo for debut performances! Farrah Fawcett plays a dumb blonde; she 

__Batch Labels Example:__

In [13]:
print(example_batch_labels[:3])

[0 1 0]


## Create the text encoder

In [15]:
VOCAB_SIZE = 1000

encoder_layer = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE)
encoder_layer.adapt(train_dataset.map(lambda text, label: text))

AttributeError: module 'tensorflow.keras.layers' has no attribute 'TextVectorization'

In [ ]:
vocab = np.array(encoder_layer.get_vocabulary())

print(vocab[:15])

In [ ]:
encoded_example = encoder_layer(example)[:3].numpy()

print(encoded_example)

In [ ]:
for n in range(2):
    print("Original: ", example[n].numpy())
    print("--------------------------------")
    print("Round-trip: ", " ".join(vocab[encoded_example[n]]))
    print()

## Create the model

In [ ]:
embedding_layer = tf.keras.layers.Embedding(
    input_dim=len(encoder_layer.get_vocabulary()),
    output_dim=64,
    mask_zero=True)

In [ ]:
model = tf.keras.Sequential([
    encoder_layer,
    embedding_layer,
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [ ]:
print([layer.supports_masking for layer in model.layers])

### Predict on a sample text without padding

In [ ]:
sample_text = ('The movie was cool. The animation and the graphics '
               'were out of this world. I would recommend this movie.')

predictions = model.predict(np.array([sample_text]))
print(predictions[0])

### Predict on a sample text with padding

In [ ]:
padding = "the " * 2000
predictions = model.predict(np.array([sample_text, padding]))
print(predictions[0])

### Compile the Keras model to configure the training process

In [ ]:
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(1e-4),
    metrics=['accuracy']
)

## Train the model

In [ ]:
history = model.fit(
    train_dataset, 
    epochs=10,
    validation_data=test_dataset,
    validation_steps=30)

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

In [ ]:
plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plot_graphs(history, 'accuracy')
plt.ylim(None, 1)

plt.subplot(1, 2, 2)
plot_graphs(history, 'loss')
plt.ylim(0, None)

### Prediction on a new sentence

In [ ]:
ample_text = ('The movie was cool. The animation and the graphics were out of this world. I would recommend this movie.')
predictions = model.predict(np.array([sample_text]))

print(predictions)

## Stack more two LSTM Layers

Set _return_sequences=True_ to generate sequences of successive outputs for each timestep (a 3D tensor of shape (batch_size, timesteps, output_features)).

In [ ]:
many_outputs_model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(len(encoder.get_vocabulary()), 64, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])

In [ ]:
many_outputs_model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(1e-4),
    metrics=['accuracy']
)

In [ ]:
many_outputs_history = model.fit(train_dataset, 
                                 epochs=10,
                                 validation_data_set=test_dataset,
                                 validation_steps=30)

In [ ]:
many_outputs_test_loss, many_outputs_test_acc = many_outputs_history.evaluate(test_dataset)

print('Test Loss:', many_outputs_test_loss)
print('Test Accuracy:', many_outputs_test_acc)

In [ ]:
sample_text = ("The movie was not good. The animation and the graphics were terrible. I would not recommend this movie.")
predictions = model.predict(np.array([sample_text]))
print(predictions)

In [ ]:
plt.figure(figsize=(16, 6))
plt.subplot(1, 2, 1)
plot_graphs(many_outputs_history, 'accuracy')

plt.subplot(1, 2, 2)
plot_graphs(many_outputs_history, 'loss')